In [1]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [2]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [3]:
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self): #ИЗМЕНЕНИЕ
        prediction = np.mean(self.labels)
        return prediction   

In [4]:
def mse(array):
    mean = array.mean()
    return np.mean((array - mean)**2)

In [5]:
def gain(left_labels, right_labels, root_criterion, criterion):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_criterion - p * criterion(left_labels) - (1 - p) * criterion(right_labels)

In [6]:
def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [7]:
def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 3

    root_mse = mse(labels) #ИЗМЕНЕНИЕ

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 3 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_labels, false_labels, root_mse, mse)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [8]:
def build_tree(data, labels):

    gain, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)

    false_branch = build_tree(false_data, false_labels)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [9]:
def calkulate_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return calkulate_object(obj, node.true_branch)
    else:
        return calkulate_object(obj, node.false_branch)

In [10]:
def predict(data, tree):
    
    answers = []
    for obj in data:
        prediction = calkulate_object(obj, tree)
        answers.append(prediction)
    return answers

In [11]:
#ИЗМЕНЕНИЕ
def err(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [12]:
regression_data, regression_labels = make_regression(n_features=2, n_informative=2, random_state=1)

In [13]:
train_data, test_data, train_labels, test_labels = train_test_split(regression_data, 
                                                                    regression_labels, 
                                                                    test_size=0.3,
                                                                    random_state=1)

In [14]:
my_tree = build_tree(train_data, train_labels)
train_answers = predict(train_data, my_tree)
answers = predict(test_data, my_tree)

In [15]:
err(train_labels, train_answers)

173.18393453897852

In [16]:
err(test_labels, answers)

410.22958949612877